In [1]:
import os
from os.path import join

import dask.dataframe as dd
import dask.array as da
import numpy as np
import pandas as pd


from dask_ml.decomposition import IncrementalPCA

In [2]:
PATH = '/lustre/scratch/users/felix.fischer/merlin_cxg_simple_norm_parquet'

In [3]:
def get_count_matrix(ddf):
    x = (
        ddf['X']
        .map_partitions(
            lambda xx: pd.DataFrame(np.vstack(xx.tolist())), 
            meta={col: 'f4' for col in range(19357)}
        )
        .to_dask_array(lengths=[1024] * ddf.npartitions)
    )

    return x


In [4]:
os.makedirs(join(PATH, 'pca'), exist_ok=True)

for split in ['test', 'val', 'train']:
    
    x = get_count_matrix(dd.read_parquet(join(PATH, split)))
    pca = IncrementalPCA(n_components=64, iterated_power=3)
    x_pca = da.compute(pca.fit_transform(x))[0]
    
    with open(join(PATH, 'pca', f'x_pca_{split}.npy'), 'wb') as f:
        np.save(f, x_pca)


/home/icb/felix.fischer/Programs/anaconda3/envs/dask/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/icb/felix.fischer/Programs/anaconda3/envs/dask/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/icb/felix.fischer/Programs/anaconda3/envs/dask/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
